<a href="https://colab.research.google.com/github/MegaDewi2/SVM-For-Text-Classification/blob/main/SVM_for_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('/content/dataset_tweet_sentimen_tayangan_tv.csv')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rizalespe/Dataset-Sentimen-Analisis-Bahasa-Indonesia/master/dataset_tweet_sentimen_tayangan_tv.csv')
df.head()

,Id,Sentiment,Acara TV,Jumlah Retweet,Text Tweet
0,1,positive,HitamPutihTransTV,12,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,2,positive,HitamPutihTransTV,6,Selamat berbuka puasa Semoga amal ibadah hari ...
2,3,positive,HitamPutihTransTV,9,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,4,positive,HitamPutihTransTV,2,selamat ya mas @adietaufan masuk hitamputih
4,5,positive,HitamPutihTransTV,1,Asiknya nonton Hitam Putih Trans7


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              400 non-null    int64 
 1   Sentiment       400 non-null    object
 2   Acara TV        400 non-null    object
 3   Jumlah Retweet  400 non-null    int64 
 4   Text Tweet      400 non-null    object
dtypes: int64(2), object(3)
memory usage: 15.8+ KB


In [ ]:
df = df.drop(columns=["Acara TV", "Jumlah Retweet"])

In [ ]:
df.rename(columns={"Sentiment":"label","Text Tweet":"tweet"}, inplace=True)

In [ ]:
df.head()

,Id,label,tweet
0,1,positive,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,2,positive,Selamat berbuka puasa Semoga amal ibadah hari ...
2,3,positive,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,4,positive,selamat ya mas @adietaufan masuk hitamputih
4,5,positive,Asiknya nonton Hitam Putih Trans7


In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl_encoder = LabelEncoder()
lbl_encoder.fit(df['label'])
df['label'] = lbl_encoder.transform(df['label'])

In [ ]:
pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.1 MB/s eta 0:00:00


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def preprocess_text(data):
    # punct = '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~'''
    stop = stopwords.words('indonesian')
    #Make lower
    data['tweet'] = data['tweet'].str.lower()
    #Remove stopwords
    data['tweet'] = data['tweet'].apply(lambda x:' '.join([words for words in x.split() if words not in stop]))
    #Remove punctuations
    data['tweet'] = data['tweet'].str.translate(str.maketrans("","",string.punctuation))
    #stemming
    data['tweet'] = [stemmer.stem(w.lower()) for w in data['tweet'] if w not in string.punctuation]
    return data
df = preprocess_text(df)

In [ ]:
df.head()

,Id,label,tweet
0,1,1,undang nshanijkt48 hitamputih menang ssk jkt48...
1,2,1,selamat buka puasa moga amal ibadah ni terima ...
2,3,1,nih trans7 hitam putih dpt harga norwegia hita...
3,4,1,selamat ya mas adietaufan masuk hitamputih
4,5,1,asiknya nonton hitam putih trans7


In [ ]:
tweet = df['tweet'].values
label = df['label'].values

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tweet = vectorizer.fit_transform(tweet)

In [ ]:
from sklearn.model_selection import train_test_split
tweet_latih, tweet_test, label_latih, label_test = train_test_split(tweet, label, test_size=0.2)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear')
model.fit(tweet_latih,label_latih)

SVC(kernel='linear')

In [ ]:
pred_svm = model.predict(tweet_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

print(confusion_matrix(label_test,pred_svm))
print(classification_report(label_test,pred_svm))

[[30  4]
 [ 6 40]]
              precision    recall  f1-score   support

           0       0.83      0.88      0.86        34
           1       0.91      0.87      0.89        46

    accuracy                           0.88        80
   macro avg       0.87      0.88      0.87        80
weighted avg       0.88      0.88      0.88        80

